In [1]:
import numpy as np
import pandas as pd
from actsnfink import *
from datetime import datetime
from myutils import loop_utils as lu
from myutils import plot_utils as pu
from myutils import data_utils as du
from astropy.time import Time, TimeDelta


# Initial model

Model trained with 30 spectroscopically classified SNe light-curves. Using a subset of training set in Leoni et al. 2021

In [2]:
retrain_all_models = True
strategy = 'UncSampling'            # query strategy

In [3]:
# read train and test data
initial_train_for_loop = pd.read_csv( '../data/initial_state/' + strategy + \
                                           '/training_samples/train_after_loop.csv', 
                                           index_col=False)

test_for_loop = pd.read_csv('../data/initial_state/' + strategy + \
                                           '/test_samples/test_after_loop.csv', 
                                           index_col=False)

# select subset of test: only SNe
test_for_loop['type'].unique()
type_list = ['SN','SNII','Ia', 'SLSN-I','SNIIP', 'SNIIn', 'SLSN-II', 'SNIc', 'SNIb', 'SNIIb', 'SNI','SNIb-pec', 'SNIc-BL','SNII-pec','SNIbn']
test_for_loop = test_for_loop[test_for_loop['type'].isin(type_list)]


# train
lu.train_and_test_initial_model(initial_train_for_loop,test_for_loop,strategy)


Number of training light-curves 40
['accuracy 0.47', 'efficiency 0.26', 'purity 0.87', 'fom 0.18']


# Data

### TNS classifications within time-range of experiment

In [4]:
# read TNS labels
df_tns_w_ztfdata = du.load_TNS_Fink('../classifications/tns_fink_20230817.parquet')
dic_allTNS_labels = lu.convert_DF_dic_labels(df_tns_w_ztfdata)

# classified by ZTF
ztf_tns = df_tns_w_ztfdata[df_tns_w_ztfdata['reporting_group']=='ZTF']
print(f"ZTF detected and spectroscopically classified {len(ztf_tns)} (not necessarily all have enough epochs for classification)")
dic_ztf_labels = lu.convert_DF_dic_labels(ztf_tns)


### Fink Siding Spring Observatory follow-up 
Fink programme to acquire labels for those candidates around P_Ia ~0.5

In [7]:
# read labels
df_Fink_AL = pd.read_csv('../classifications/ALbot_with_classifications_curated_original.tsv', sep='\t')

df_Fink_AL['fup_jd'] = [Time(datetime.strptime(d[0], '%d/%m/%Y')).jd for d in df_Fink_AL[['Appears in bot']].values]
df_Fink_AL['tmp'] = [datetime.strptime(d[0], '%d/%m/%Y') for d in df_Fink_AL[['Appears in bot']].values]
df_Fink_AL['fup_jd_strfmt'] = df_Fink_AL.loc[:,'tmp'].apply(lambda x: du.add_days_and_format(Time(x), 0))

# some of the Fink AL identified objects might have been classified by another group and reported to TNS
print(f"Adding {len(df_Fink_AL[df_Fink_AL['ztf_names'].isin(df_tns_w_ztfdata['ztf_names']) & df_Fink_AL['Classification'].isna()])} \
      classifications from TNS to Fink AL")

# Fill classification with TNS type if missing
df_merged = pd.merge(df_Fink_AL, df_tns_w_ztfdata, how='left', on='ztf_names')
df_merged['Classification'] = df_merged.apply(lambda row: row.type if row.Classification is None else row.Classification, axis=1)
# Drop the 'type' column since it's now renamed
df_merged = df_merged.drop(columns=['type'])
# keeep only those with classification
df_Fink_AL_w_classifications = df_merged[df_merged['Classification'].notna()]
df_Fink_AL_w_classifications.reset_index(drop=True, inplace=True)

# reformat type for AL loop
df_Fink_AL_w_classifications['type AL'] = df_Fink_AL_w_classifications.loc[:,'Classification'].apply(lambda x: 'Ia' if 'Ia' in x else 'other')

# sort
df_Fink_AL_w_classifications = df_Fink_AL_w_classifications.sort_values(by='fup_jd')

# convert to AL format
new_labels = lu.convert_DF_dic_labels(df_Fink_AL_w_classifications, key_spectra='fup_jd')

Adding 0       classifications from TNS to Fink AL


In [8]:
if retrain_all_models:
    metrics_fink_all = lu.AL_loop(new_labels,strategy, test_for_loop, dir_suffix ="fink", proba_cut = False, plot_lcs=False,verbose=True)
else:
    metrics_fink_all = pd.read_csv('../dump/data_fink/' + "metrics.csv")
    

20230924
Read initial state
New alerts to use 1
    train_updated.shape =  (41, 15)
   res =  [np.float64(0.4478632478632479), np.float64(0.21865154379332072), np.float64(0.8696741854636592), np.float64(0.15083913657312584)]
20230925
Read 20230924 state
New alerts to use 1
    train_updated.shape =  (42, 15)
   res =  [np.float64(0.4478632478632479), np.float64(0.2180214240705734), np.float64(0.871536523929471), np.float64(0.15117317180043768)]
20230926
Read 20230925 state
Missing 0    ZTF23abceaws
Name: id, dtype: object
             id   type           MJD FLT        FLUXCAL    FLUXCALERR
0  ZTF23abceaws  other  2.460213e+06   g   39084.449559   2052.957962
1  ZTF23abceaws  other  2.460210e+06   g   15908.484165   2119.210286
2  ZTF23abceaws  other  2.460207e+06   g   68928.749559   2548.997318
3  ZTF23abceaws  other  2.460205e+06   g   46760.591915   1573.235369
4  ZTF23abceaws  other  2.460203e+06   g   29246.910762   1271.715389
5  ZTF23abceaws  other  2.460201e+06   g   31062.757

## All ZTF reported classifications (from TNS)

In [9]:
# Function to convert Time object to desired format
def add_days_and_format(x,days):
    time_obj = Time(x, format='iso') + TimeDelta(days, format='jd')
    dt = time_obj.datetime 
    return f"{dt.year}{dt.month:02d}{dt.day:02d}"

In [10]:
if retrain_all_models:
    metrics_ztf = lu.AL_loop(dic_ztf_labels, strategy, test_for_loop, dir_suffix='ZTF', proba_cut = False,verbose=False, plot_lcs=False)
else:
    metrics_ztf = pd.read_csv('../dump/data_ZTF/' + "metrics.csv")

Read initial state
Read 20231006 state
Read 20231016 state
Read 20231017 state
Read 20231017 state
Read 20231021 state
Read 20231022 state
Read 20231025 state
Read 20231026 state
Read 20231026 state
Read 20240408 state
Read 20240408 state
Read 20240408 state
Read 20240415 state
Read 20240416 state
Read 20240419 state
Read 20240420 state
Read 20240420 state
Read 20240420 state
Read 20240424 state
Read 20240425 state
Read 20240426 state
Read 20240428 state
Empty data for ZTF24aajznoi or error in request <Response [200]>
Empty data for ZTF18aauotcv or error in request <Response [200]>
Read 20240430 state
Read 20240501 state
Read 20240502 state
Empty data for ZTF24aakrsww or error in request <Response [200]>
Read 20240503 state
Read 20240503 state
Read 20240503 state
Read 20240510 state
Read 20240511 state
Read 20240511 state
Read 20240511 state
Read 20240514 state
Read 20240515 state
Read 20240515 state
Read 20240515 state
Empty data for ZTF24aamjepg or error in request <Response [200]>
E

### all labels in literature with ZTF data


In [11]:
if retrain_all_models:
    metrics_allTNS = lu.AL_loop(dic_allTNS_labels, strategy, test_for_loop, dir_suffix='allTNS', proba_cut = False,verbose=False, plot_lcs=False)
else:
    metrics_allTNS = pd.read_csv('../dump/data_allTNS/' + "metrics.csv")


Read initial state
Empty data for ZTF23abglsmj or error in request <Response [502]>
No previous date found
Read initial state
Read 20231004 state
Read 20231004 state
Read 20231006 state
Read 20231007 state
Read 20231008 state
Read 20231009 state
Read 20231011 state
Read 20231011 state
Read 20231014 state
Empty data for ZTF23abmsmzq or error in request <Response [200]>
Read 20231015 state
Read 20231016 state
Read 20231017 state
Read 20231018 state
Read 20231019 state
Read 20231020 state
Read 20231021 state
Read 20231022 state
Read 20231023 state
Read 20231025 state
Read 20231025 state
Read 20231025 state
Read 20231025 state
Read 20231025 state
Read 20240408 state
Read 20240409 state
Read 20240410 state
Read 20240410 state
Read 20240413 state
Read 20240414 state
Read 20240414 state
Read 20240419 state
Read 20240419 state
Read 20240419 state
Read 20240423 state
Read 20240424 state
Read 20240425 state
Read 20240426 state
Read 20240427 state
Empty data for ZTF24aajzmzf or error in request <

### Analysis

In [12]:
# date reformatting to align with all dates in our metrics
all_dates = np.concat([metrics_fink_all['date'].astype(int).values,metrics_allTNS['date'].astype(int).values, metrics_ztf['date'].astype(int).values])
all_dates = np.unique(all_dates)

# reformatting
metrics_fink_all['date'] = metrics_fink_all['date'].astype(int)
# lets put the date_plot in the same baseline using the combining dataset
to_merge = pd.DataFrame({'date':all_dates, 'date_plot_universal':np.arange(0,len(all_dates))})
metrics_fink_all_plot = pd.merge(metrics_fink_all, to_merge, on='date',how='left')
metrics_ztf['date'] = metrics_ztf['date'].astype(int)
metrics_ztf_plot = pd.merge(metrics_ztf, to_merge, on='date',how='left')
metrics_allTNS['date'] = metrics_allTNS['date'].astype(int)
metrics_allTNS_plot = pd.merge(metrics_allTNS, to_merge, on='date',how='left')

In [1]:
# not completely true as ZTF spectra dates are different than Fink AL ones
# pu.plot_metrics_listdf([metrics_fink_all_plot,metrics_ztf_plot, metrics_allTNS_plot],['Fink AL + ANU2.3m follow-up','ZTF spectra', 'TNS'],varx='date_plot_universal', plots_dir = '../plots/')

In [2]:
# pu.plot_metrics_listdf([metrics_fink_all_plot,metrics_ztf_plot, metrics_allTNS_plot],['Fink AL + ANU2.3m follow-up','ZTF spectra', 'TNS'],varx='n spectra', plots_dir = '../plots/')

In [15]:
import plotly.graph_objects as go

# Create the first scatter plot trace
trace1 = go.Scatter(x=metrics_fink_all['n spectra'].values, y=metrics_fink_all['accuracy'].values, mode='markers', name='Fink', marker=dict(color='blue'),hovertext=metrics_fink_all['query_objectIds'].astype(str).values, hoverinfo='text+x+y')
# Create the second scatter plot trace
trace2 = go.Scatter(x=metrics_ztf['n spectra'].values, y=metrics_ztf['accuracy'].values, mode='markers', name='ZTF', marker=dict(color='orange'),hovertext=metrics_ztf['query_objectIds'].astype(str).values, hoverinfo='text+x+y')

trace3 = go.Scatter(x=metrics_allTNS['n spectra'].values, y=metrics_allTNS['accuracy'].values, mode='markers', name='TNS', marker=dict(color='grey'),hovertext=metrics_allTNS['query_objectIds'].astype(str).values, hoverinfo='text+x+y')

# Create the figure and add the two traces
fig = go.Figure(data=[trace1, trace2, trace3])
# Update layout to set the x-axis label
fig.update_layout(
    xaxis_title='n spectra',  # Set x-axis label
    yaxis_title='accuracy'   # Set y-axis label (optional)
)
# Show the figure
fig.show()
